In [1]:
import sys
sys.path.append('..')
sys.path.append('/Users/jh7685/Documents/Projects/pysurfer')
from pysurfer.mgz_helper import map_values_as_mgz
from pysurfer.mgz_helper import load_mgzs
import os
import numpy as np
import warnings
from itertools import product
warnings.filterwarnings("ignore", category=UserWarning)

%load_ext autoreload
%autoreload 2

# Configurations

In [33]:
nsd_dir ='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset'
output_dir='/Volumes/server/Projects/sfp_nsd/derivatives'
fs_subj_dir='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer'
overlay_dir='/Volumes/server/Projects/sfp_nsd/derivatives/sfp_maps/mgzs/nsdsyn'
fig_dir='/Volumes/server/Projects/sfp_nsd/derivatives/figures/brain_maps/nsdsyn'

sn_list = [f'subj{sn:02d}' for sn in np.arange(1,9)]

# Make mask out of them

In [ ]:
threshold=6
for sn, hemi in product(sn_list, ['lh', 'rh']):
    f_path = os.path.join(overlay_dir, f'{hemi}.sub-{sn}_value-precision.mgz')
    precision_mask = load_mgzs(f_path, fdata_only=True, squeeze=False)
    precision_mask[precision_mask < threshold] = 0
    precision_mask[precision_mask > threshold] = 1
    save_path = os.path.join(overlay_dir, f'{hemi}.sub-{sn}_value-precision_masked.mgz')
    map_values_as_mgz(template=f_path, data=precision_mask, save_path=save_path)

# Multiply mask and sfp maps

In [32]:
threshold=6
for sn, hemi in product(sn_list, ['lh', 'rh']):
    for my_val in ['mode', 'amp', 'sigma']:
        f_path = os.path.join(overlay_dir, f'{hemi}.sub-{sn}_value-precision.mgz')
        precision_mask = load_mgzs(f_path, fdata_only=True, squeeze=False)
        mode_map = load_mgzs(os.path.join(overlay_dir, f'{hemi}.sub-{sn}_value-{my_val}.mgz'), fdata_only=True, squeeze=False)
        precision_mask = np.greater(precision_mask, threshold)
        mode_map[precision_mask == False] = np.nan
        save_path = os.path.join(overlay_dir, f'{hemi}.sub-{sn}_value-{my_val}_masked.mgz')
        map_values_as_mgz(template=f_path, data=mode_map, save_path=save_path)


NameError: name 'sn_list' is not defined

# Load spatial frequency preference (peak) maps for all subjects
The transformation to fsaverage space should be done in the .sh script

In [2]:
subj_list = [f'subj{i:02d}' for i in range(1, 9)]
val_list = ['mode', 'amp', 'sigma']

nsd_dir ='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset'
output_dir='/Volumes/server/Projects/sfp_nsd/derivatives'
fs_subj_dir='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer'
overlay_dir='/Volumes/server/Projects/sfp_nsd/derivatives/sfp_maps/mgzs/nsdsyn'
fig_dir='/Volumes/server/Projects/sfp_nsd/derivatives/figures/brain_maps/nsdsyn'

# Average them & save the averaged sfp map

In [23]:
all_mgzs = {}
for hemi, my_val in product(['lh', 'rh'], val_list[:1]):
    tmp_mgzs = []
    for subj in subj_list:
        f_path = os.path.join(overlay_dir, f'{hemi}.sub-{subj}_value-{my_val}_space-fsaverage_masked.mgz')
        tmp_mgz = load_mgzs(f_path, fdata_only=True, squeeze=True)
        tmp_mgzs.append(tmp_mgz)
    k = f'{hemi}-{my_val}'
    tmp_mgzs = np.asarray(tmp_mgzs)
    all_mgzs[k] = np.nanmean(tmp_mgzs, axis=0)

/Users/jh7685/opt/miniconda3/envs/sfp/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


In [31]:
for hemi, my_val in product(['lh', 'rh'], val_list[:0]):
    k = f'{hemi}-{my_val}'
    f_path = os.path.join(overlay_dir, f'{hemi}.sub-subj01_value-{my_val}_space-fsaverage_masked.mgz')
    save_path = os.path.join(overlay_dir, f'{hemi}.sub-fsaverage_value-{my_val}_space-fsaverage_masked.mgz')
    map_values_as_mgz(f_path, all_mgzs[k], save_path)